In [1]:
%matplotlib inline
import pandas as pd

import os

# Load in SRA metadata and search results

### Load in SRA metadata info:

This is the 'run info' that you can download from NCBI in bulk; it's got one entry for every accession, approximately.

In [2]:
run_info = pd.read_csv('../big.runinfo.csv.gz')

/var/folders/1c/s23lyl9x7cgd1rg4n6t867qr0000gn/T/ipykernel_10136/1351821980.py:1: DtypeWarning: Columns (8,33,38,39,40,43) have mixed types. Specify dtype option on import or set low_memory=False.
  run_info = pd.read_csv('../big.runinfo.csv.gz')


The two most important columns for our purposes are 'Run' and 'ScientificName':

In [3]:
print(run_info.columns)

run_info[['Run', 'ScientificName']].head()

Index(['Run', 'ReleaseDate', 'LoadDate', 'spots', 'bases', 'spots_with_mates',
       'avgLength', 'size_MB', 'AssemblyName', 'download_path', 'Experiment',
       'LibraryName', 'LibraryStrategy', 'LibrarySelection', 'LibrarySource',
       'LibraryLayout', 'InsertSize', 'InsertDev', 'Platform', 'Model',
       'SRAStudy', 'BioProject', 'Study_Pubmed_id', 'ProjectID', 'Sample',
       'BioSample', 'SampleType', 'TaxID', 'ScientificName', 'SampleName',
       'g1k_pop_code', 'source', 'g1k_analysis_group', 'Subject_ID', 'Sex',
       'Disease', 'Tumor', 'Affection_Status', 'Analyte_Type',
       'Histological_Type', 'Body_Site', 'CenterName', 'Submission',
       'dbgap_study_accession', 'Consent', 'RunHash', 'ReadHash'],
      dtype='object')


,Run,ScientificName
0,SRR18036904,bovine metagenome
1,SRR18036905,bovine metagenome
2,SRR18036906,bovine metagenome
3,SRR18036907,bovine metagenome
4,SRR18036908,bovine metagenome


In [4]:
# there are ~700,000 entries:
len(run_info)

702013

In [5]:
run_info2 = run_info[['Run', 'ScientificName']]

### Now, load in the stamps MAGsearch results


In [6]:
magsearch_df = pd.read_csv('../output.magsearch.k31/results/stamps.csv', quotechar="'")
print(len(magsearch_df))
magsearch_df.head()

5527


,query,Run,containment
0,FV_DSM_15829_genome,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.038168
1,FV_DSM_15829_genome,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.091603
2,FV_PB189-T1-4_genome,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.037313
3,FV_DSM_15829_genome,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.351145
4,FV_PB189-T1-4_genome,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.149254


# Make the results more human-readable and add SRA metadata info

### First, we need to take the filenames in the 'Run' column and turn them into accessions.

In [7]:
def extract_run_acc(x):
    # get just the end filename
    x = os.path.basename(x)
    # remove extension '.sig'
    y, ext = os.path.splitext(x)
    assert ext == '.sig', ext
    return y

# this can be used in case we have .gz, .fasta, .fa, etc in the query filename
def remove_extension(x):
    x = os.path.basename(x)
    y, ext = os.path.splitext(x)
    while ext in ('.gz', '.fasta', '.fa', '.fna'):
        x = y
        y, ext = os.path.splitext(x)
    return y

magsearch_df['Run'] = magsearch_df['Run'].apply(extract_run_acc)
magsearch_df.head()

,query,Run,containment
0,FV_DSM_15829_genome,ERR4333983,0.038168
1,FV_DSM_15829_genome,SRR17547361,0.091603
2,FV_PB189-T1-4_genome,SRR17547361,0.037313
3,FV_DSM_15829_genome,ERR2014370,0.351145
4,FV_PB189-T1-4_genome,ERR2014370,0.149254


### Now we can correlate magsearch results with SRA RunInfo

In [8]:
run_info2.set_index('Run')#['ScientificName']

,ScientificName
Run,
SRR18036904,bovine metagenome
SRR18036905,bovine metagenome
SRR18036906,bovine metagenome
SRR18036907,bovine metagenome
SRR18036908,bovine metagenome
...,...
SRR11108097,gut metagenome
SRR8144073,Uvigerina striata
SRR8144074,Uvigerina striata


In [9]:
magsearch2_df = magsearch_df.set_index('Run').join(run_info2.set_index('Run')['ScientificName'])
magsearch2_df.head()

,query,containment,ScientificName
Run,,,
DRR042391,FV_DSM_15829_genome,0.030534,NaN
DRR042545,FV_DSM_15829_genome,0.022901,NaN
DRR042550,FV_DSM_15829_genome,0.022901,NaN
DRR042658,FV_DSM_15829_genome,0.015267,NaN
DRR086622,S26,0.020896,marine metagenome


### Subset to just SRA results with good scientific names

In [10]:
# how many have 'null' scientific name?
null_df = magsearch2_df[magsearch2_df['ScientificName'].isnull()]
print(len(null_df))

601


In [11]:
# pull out just the ones with good scientific names:
magsearch3_df = magsearch2_df[~magsearch2_df['ScientificName'].isnull()]
perc_non_null = len(magsearch3_df)/len(magsearch2_df)*100
print(f"Of {len(magsearch2_df)} MAGsearch results, {len(magsearch3_df)} have non-null metadata ({perc_non_null:.2f}%)")
magsearch3_df.head()

Of 5527 MAGsearch results, 4926 have non-null metadata (89.13%)


,query,containment,ScientificName
Run,,,
DRR086622,S26,0.020896,marine metagenome
DRR086623,S26,0.018140,marine metagenome
DRR086624,S26,0.019977,marine metagenome
DRR086640,S26,0.017451,marine metagenome
DRR086642,S26,0.024569,marine metagenome


In [12]:
print(f'{len(set(magsearch3_df["query"]))} independent queries in results')

4 independent queries in results


In [13]:
# how many matches do we have for each query?
magsearch3_df["query"].value_counts()#[:20]

FV_DSM_15829_genome     3083
FV_PB189-T1-4_genome     890
S26                      841
C3_T13_0                 112
Name: query, dtype: int64

### Split Results by Query

We two very different queries!  Let's split our results into a dataframe that only contains the marine queries.

marine_queries = ["S26", "C3_T13_0"]

vg_queries = ["FV_DSM_15829_genome", "FV_PB189-T1-4_genome"]

In [14]:
#marine_queries = ["S26", "C3_T13_0"]
vg_queries = ["FV_DSM_15829_genome", "FV_PB189-T1-4_genome"]

#marine_df = magsearch3_df[magsearch3_df["query"].isin(marine_queries)]
vg_df = magsearch3_df[magsearch3_df["query"].isin(vg_queries)]

In [15]:
vg_df["query"].unique()

array(['FV_DSM_15829_genome', 'FV_PB189-T1-4_genome'], dtype=object)

In [16]:
vg_df.head()

,query,containment,ScientificName
Run,,,
DRR127774,FV_DSM_15829_genome,0.137405,human gut metagenome
DRR162207,FV_DSM_15829_genome,0.053435,human gut metagenome
DRR162212,FV_DSM_15829_genome,0.022901,human gut metagenome
DRR171463,FV_DSM_15829_genome,0.236641,human gut metagenome
DRR171463,FV_PB189-T1-4_genome,0.123134,human gut metagenome


# Start looking at the results!


In [17]:
# what are the top ScientificNames of the matches?
vg_df["ScientificName"].value_counts()[:20]

human gut metagenome                    728
human vaginal metagenome                650
human metagenome                        580
human skin metagenome                   467
metagenome                              399
wastewater metagenome                   211
vaginal metagenome                      203
gut metagenome                          155
air metagenome                          108
Homo sapiens                             78
skin metagenome                          46
urine metagenome                         38
Chlamydia trachomatis                    36
indoor metagenome                        32
feces metagenome                         29
unidentified                             20
human lung metagenome                    19
human urinary tract metagenome           17
human reproductive system metagenome     14
Neisseria gonorrhoeae                    13
Name: ScientificName, dtype: int64

## Sort Results by Containment

The default threshold for containment is 0.01, which means ~1% of the query genome needs to be found in the metagenome for it to be reported. That's not very stringent!

First, let's look at the SRA runs that had the **best** containment of our queries:

In [18]:
vg_df.sort_values(by=['containment'], ascending=False)[:20]

,query,containment,ScientificName
Run,,,
SRR063897,FV_PB189-T1-4_genome,1.000000,human metagenome
SRR17635737,FV_DSM_15829_genome,0.862595,human vaginal metagenome
SRR16916853,FV_DSM_15829_genome,0.854962,human metagenome
SRR17635738,FV_DSM_15829_genome,0.847328,human vaginal metagenome
SRR16916862,FV_DSM_15829_genome,0.832061,human metagenome
SRR17635740,FV_DSM_15829_genome,0.824427,human vaginal metagenome
SRR17635741,FV_DSM_15829_genome,0.816794,human vaginal metagenome
SRR513452,FV_DSM_15829_genome,0.809160,human metagenome
SRR16916861,FV_DSM_15829_genome,0.778626,human metagenome


## Filter Results by Containment

We've found (rule of thumb) that 0.2 is a decent value - 20% - indicating some level of stringency. Let's take a look -

In [19]:
# let's do some filtering -
vg_df2 = vg_df[vg_df['containment'] > 0.2]

for name, df in {"vaginal": vg_df2}.items():
    print('query type:', name)
    print('total matches:', len(df))
    print('query:', len(set(df["query"])))
    print('metagenomes:', len(set(df.index)))
    print("\n")
    print(df["ScientificName"].value_counts()[:20], "\n\n")

query type: vaginal
total matches: 948
query: 2
metagenomes: 806


human vaginal metagenome                338
human metagenome                        242
human gut metagenome                     93
vaginal metagenome                       70
human skin metagenome                    62
metagenome                               41
Homo sapiens                             30
urine metagenome                         14
gut metagenome                           11
human reproductive system metagenome      9
human lung metagenome                     8
Chlamydia trachomatis                     8
skin metagenome                           5
indoor metagenome                         4
human urinary tract metagenome            2
feces metagenome                          2
urinary tract metagenome                  2
human eye metagenome                      2
Human papillomavirus                      1
human oral metagenome                     1
Name: ScientificName, dtype: int64 




## Now let's filter at >50%

Especially for our vg query, we have a _lot_ of results. Let's be a bit more stringent.

In [20]:
vg_df3 = vg_df[vg_df['containment'] > 0.5]

for name, df in {"vaginal": vg_df3}.items():
    print('query type:', name)
    print('total matches:', len(df))
    print('query:', len(set(df["query"])))
    print('metagenomes:', len(set(df.index)))
    print("\n")
    print(df["ScientificName"].value_counts()[:20], "\n\n")

query type: vaginal
total matches: 336
query: 2
metagenomes: 303


human vaginal metagenome                149
human metagenome                        129
human gut metagenome                     16
vaginal metagenome                       14
Homo sapiens                              8
human reproductive system metagenome      3
urine metagenome                          3
human skin metagenome                     3
human lung metagenome                     3
skin metagenome                           2
feces metagenome                          1
Human papillomavirus                      1
urinary tract metagenome                  1
hydrothermal vent metagenome              1
money metagenome                          1
indoor metagenome                         1
Name: ScientificName, dtype: int64 




In [21]:
# save these results to a file
vg_df3.to_csv("vg_metagenome_results.k31.0.5.containment.csv")

In [22]:
# if you want, you can subset by SRA metadata..
vg_only_df = vg_df3[vg_df3["ScientificName"] == 'human vaginal metagenome']
vg_only_df["query"].value_counts()#[:20]

FV_DSM_15829_genome     103
FV_PB189-T1-4_genome     46
Name: query, dtype: int64